In [1]:
from utils.save_read_graph import read, save
from section3.scenario1 import scenario_1


In [10]:
data_path = './data/'
network_path = './networks/p400R400000/'
cn, x, _= read(network_path)
x = {int(k): v for k, v in x.items() }
scenario_1(cn, x)


In [31]:
import networkx as nx
from tqdm.auto import tqdm
from ortools.linear_solver import pywraplp
def prepare_data(cn, prefix='S3P1'):
    n_candidate_site = len(cn.nodes)

    clients = [k for k, v in dict(cn.nodes(data=prefix+"_station_size")).items() if v > 0] 
    print('Computing shortest path...')
    d = {i: { j: nx.path_weight(cn, nx.shortest_path(cn, j, i), 'weight')   for j in range(n_candidate_site)  if i!= j} for i in tqdm(clients)}
    a = {k:v  for k, v in dict(cn.nodes(data=prefix+'_h2day')).items() if k in clients}
    return n_candidate_site, clients, d,  a
def PMedian(cn, p_facilities):
    n_candidate_site, clients, d,  a = prepare_data(cn)

    solver = pywraplp.Solver.CreateSolver('GLOP')
    if not solver:
            return
    Y = [solver.BoolVar('y_' + str(j)) for j in range(n_candidate_site)]
    X = {i: [solver.BoolVar('x_' + str(i) + '_'+str(j))  for j in range(n_candidate_site)  ] for i in clients}
    print('Adding constraints...')
    for i in tqdm(X):
        solver.Add(sum(X[i]) == 1)
        for j in range(len(X[i])):
            solver.Add(X[i][j] <= Y[j])
    solver.Add(sum(Y) == p_facilities)
    solver.Minimize(sum([sum([ a[i]* d[i][j] *X[i][j] for j in d[i]]) for i in clients]))
    status = solver.Solve()

    print('\nAdvanced usage:')
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print()
    # print(sum([sum([ a[i]* d[i][j] *X[i][j].solution_value() for j in d[i]]) for i in clients]))
    return {k: [ v[j].solution_value() for j in range(len(v))] for k, v in X.items()}, [ele.solution_value() for ele in Y]





In [27]:
X, Y = PMedian(cn, 80)

Computing shortest path...


100%|██████████| 335/335 [04:20<00:00,  1.29it/s]


Adding constraints...


100%|██████████| 335/335 [00:04<00:00, 80.02it/s]



Advanced usage:
Problem solved in 21019.000000 milliseconds
Problem solved in 39652 iterations



In [30]:
set(Y)


{0.0, 0.5, 1.0}